Nom Etudiant 1: <font color='blue'>NOM Prénom</font>
<br>
Nom Etudiant 2: <font color='blue'>NOM Prénom</font>

# TME 4 : Recheche de pattern (motifs) en utilisant les Suffix Trees

Un suffix tree est construit à partir d'un jeu de séquences. Une fois construit, il permet de rechercher un motif en temps $O(k)$ où $k$ est la longueur du motif.

Dans un premier temps, nous allons finaliser la mise en place de la création d’une liste de k-mers filtrés. Pour cela, nous allons développer une fonction permettant de générer automatiquement une liste de motifs filtrés (**Partie A**).

Ensuite, nous réutiliserons les séquences artificielles des TMEs précédents, dans lesquelles les motifs ont été implantés avec ou sans variation. Nous implémenterons l'algorithme suffix tree afin de rechercher efficacement ces motifs, d’abord dans leur version exacte (**Partie B**), puis en tenant compte des variations (**Partie C**).


## Partie A : Fonction de génération de la liste de k-mers

1\. Avant de rechercher les motifs, implémentez ou réutilisez les fonctions permettant de générer tous les k-mers possibles de taille k, en éliminant les motifs peu complexes afin d'éviter des calculs inutiles. Créez une fonction `createKmers` pour automatiser cette tâche. Sauvegardez la liste générée dans un fichier afin de pouvoir la réutiliser ultérieurement.

In [1]:
# Importer fonctions
from TME import *
from itertools import *

def createKmers(k, m, n, p, variation):
    """
    entrée k : taille de kmers
    entrée m : taille de repetition de nucléotide
    entrée n: taille de répétition de dinucléotide
    entrée p: proportion de nucleótides T et A
    entrée variation : string, si True permettre variation d'un nucléotide
    sortie motifsClean: liste de motifs sans les motifs peu complexe
    """
    
    nuc = ('A', 'C', 'G', 'T')
    aux = list(product(nuc,repeat = k))
    motifs = [''.join(motif) for motif in aux]
    motifs = removeLowComplexeHomo(motifs,m)
    motifs = removeLowComplexeHetero(motifs,n,variation)
    motifs = removeTARich(motifs,p)
    
    return motifs

allkmers_k4 = createKmers(4,4,1,0.7,"no")

In [2]:
# Sauvegarder les k-mers
kmers_file = "allkmers_k4_m4_n1_p0.7_v0.txt" 

f = open(kmers_file,"w") 
f.write("\n".join(allkmers_k4))
f.write("\n")
f.close()

## Partie B : Suffix Trees Exact Matching

1\. Réutilisez les séquences artificielles des TME précédents contenant le motif sans variation, ainsi que le motif initial.

In [3]:
seqs_file = 'seqs_artificielles_impl.txt'

f = open(seqs_file,"r")
tmp = f.readlines()
seqs = []
for line in tmp :
    seqs.append(line[:-1])
f.close()

for seq in seqs:
    print(seq)

motif_file = "motif.txt"
f = open(motif_file,"r")
motif = f.readline()
f.close()

print("\nLe motif géneré était :", motif)

ACATCCCTAGAAGGATGGTCTGTTCGGATCTGTTTCACATTTAGA
GTGCCACCTACATACACACTAACCCATTGGTGGTAACCACTCTGA
CTATTCGTGTAATAACATAGACAAGACGGCTGGATCGGCTTCTCT
CGAGGGATTACCACATTGAGAGGTAGTGTTGGCCGTCGATTTAAT
GAGCTTGTGTTTATGCAGTACGTCAGACATGAAATCTCAGCCATT
TCTTGCACATTATACTCTCGTCAAGCATCAGTAGCGATGGTGTCG
GTGGCGGGATCAGGGGCACATAGTTCCTCATCTACTGCGGCAGTG
GGGTACATCCGACAACCTACCGGTCGCACCAGTTTACCCGTTATA
CGTCAGAGAGCTACCAGTAATTAACATGCCAATCACATGGAGTGT
ACATTCTTCGTCACGAGCCTTCTAGAGAAATTGATGCAGCTCGTC

Le motif géneré était : ACAT


2\. Utilisez le package Python suffix-trees, disponible ici : https://pypi.org/project/suffix-trees/ pour créer un suffix tree à partir de la chaîne de caractères ci-dessous, puis recherchez les motifs proposés.

Installation (si le package n'est pas disponible) : `!pip install suffix-trees`

In [4]:
from suffix_trees import STree

st = STree.STree("abcdefghab")

print(st.find("abc")) # 0

print(st.find_all("ab")) # {8, 0}

0
{0, 8}


3\. Définissez une fonction `constructTree` pour construire un suffix tree à partir des séquences artificielles implantées. **Attention :** Avant de créer l'arbre, concaténez vos séquences en les séparant par un espace. Testez ensuite si votre fonction est capable de retrouver le motif sans variation implanté.

In [5]:
# Concatener les séquences
seqs_concat = " ".join(seqs)
seqs = seqs_concat

In [6]:
def constructTree(sequences):
    """
    construis un abre de suffixes
    entrée sequences : liste de séquences
    sortie suffix_tree : arbre de suffixes
    """    

    suffix_tree = STree.STree(sequences)

    return suffix_tree

tree = constructTree(seqs)

fix_motif = 'ACAT' # changer pour un teste rapide

print(tree.find_all(fix_motif))

{0, 36, 293, 326, 391, 106, 236, 402, 210, 150, 55, 414}


4\. Définissez la fonction `exactMatch` qui parcourt le suffix tree pour rechercher tous les motifs possibles (k-mers) générés à la question précédente. La fonction doit renvoyer un dictionnaire contenant les motifs (clés) et leur nombre d'occurrences (valeurs). Ensuite, triez,  rassemblez, et affichez les 10 motifs les plus fréquents dans votre jeu de données artificiel.

In [7]:
def exactMatch(kmersV, stree):
    """
    Cherche dans le suffix tree tous les motifs possibles
    entrée kmersV: liste de Kmers à chercher
    entrée stree: suffix tree
    sortie motif_occur : dictionnaire qui contient les motifs (clés) et leurs nombre d'occurrences (values).
    """

    motif_occur = {k:len(stree.find_all(k)) for k in kmersV}
    
    return motif_occur

motif_occur_dict = exactMatch(allkmers_k4, tree)

In [8]:
# Trier dictionnaire
motif_occur_dict = getTopMotifs(motif_occur_dict,20,True)

# Afficher top 10
for x in motif_occur_dict : print(x,":",motif_occur_dict[x])

TCAG : 5
CAGT : 5
GGAT : 5
CATC : 4
CTAC : 4
GTCA : 4
TACC : 4
GATG : 3
ATGC : 3
CATG : 3
AGCT : 3
GGTA : 3
GATC : 3
CCTA : 3
TGCA : 3
ATGG : 3
CTAG : 2
CCAT : 2
ATCC : 2
GTAG : 2


In [9]:
# Rassembler dictionnaire
motif_occur_dict_gat = gatherRevCompMotifs(motif_occur_dict)

# Afficher top 10
motif_occur_dict_gat = getTopMotifs(motif_occur_dict_gat,10,True)
for x in motif_occur_dict_gat : print(x,":",motif_occur_dict[x])

GGAT : 5
TCAG : 5
CAGT : 5
CATC : 4
TACC : 4
CTAC : 4
GTCA : 4
CATG : 3
AGCT : 3
GATC : 3


## Partie C : Suffix Trees Inexact Matching

1\. Réutilisez vos séquences artificielles contenant le motif variable.

In [10]:
seqs_var_file = 'seqs_artificielles_impl_var.txt'

f = open(seqs_var_file,"r")
tmp = f.readlines()
seqs_var = []
for line in tmp :
    seqs_var.append(line[:-1].upper())
f.close()
        
for seq in seqs_var:
    print(seq)

ACATCCCTAGAAGGACAAATGGTCTGTTCGGATCTGTTTCTTAGA
GTGCCACCTACACACTAACCCAAAATTTGGTGGTAACCACTCTGA
CTATTCGTGTAATAAGACAAGACGGCTGGATCGACATGCTTCTCT
CGAGGGATTACCTGAGAGGTAGTGTTGACATGCCGTCGATTTAAT
GAGCTTGTGTTTATGCAACATGTACGTCAGGAAATCTCAGCCATT
TCTTGCTATACTCTCGTCAAGCATCAGTAGCGATGGTGTCACATG
GTGGCGGGATCAGGGGCAGTTCCTCATCACATTACTGCGGCAGTG
GGGACATTCCGACAACCTACCGGTCGCACCAGTTTACCCGTTATA
CGTCAGAGACATAGCTACCAGTAATTAACATGCCAATCGGAGTGT
TCTTCGTCACGAGCCTTACATCTAGAGAAATTGATGCAGCTCGTC


2\. Construisez à nouveau le suffix tree en utilisant ces nouvelles séquences et votre fonction `constructTree`.

In [11]:
# Concatener les séquences
seqs_var = " ".join(seqs_var)

# Construire arbre
tree_var = constructTree(seqs_var)

3\. Nous allons maintenant implémenter l'algorithme en version inexacte. Pour ce faire, utilisez la technique _seed_ afin de détecter le motif variable. À partir d’une liste de k-mers et d’un nombre de variations, déterminez la taille du seed (`seed_size`) ainsi que le nombre de seeds à utiliser (`seed_nb`).

**Attention :**

* Certaines tailles de k-mers peuvent générer des seeds incomplets (ex. `k = 5`).
* Assurez-vous de ne pas utiliser de seeds plus petits que la taille minimale requise.

In [12]:
from math import floor

def getSeeds(kmersV, v):
    """
    entrée kmersV : liste de motifs à chercher
    entrée v : nombre de variations dans les motifs
    sortie k : taille de k-mer
    sortie seed_size : taille de seed calculé
    sortie seed_nb : nombre de seeds
    """
    
    k = len(kmersV[0])
    
    seed_nb = v+1
    
    if (k%seed_nb != 0) :
        seed_size = floor(k/seed_nb) + 1
    else :
        seed_size = floor(k/seed_nb)
    
    return k, seed_size, seed_nb

getSeeds(allkmers_k4, v = 1)

(4, 2, 2)

In [13]:
kmer_test1 = ["TACA", "TTCA", "GGGG"] # perfect match, imperfect match, no match
kmer_test2 = ["TACAG", "TTCAA", "GGGGG"] # uneven kmers
kmer_test3 = ["TACAGC", "TTCAAG", "GGGGGG"] # longer kmers

v = 1

k, seed_size, seed_nb = getSeeds(kmer_test1,v)

print("kmer_test1 k = %s\n\tseed size: %s\n\tseed number: %s\n" % (k, seed_size, seed_nb))

k, seed_size, seed_nb = getSeeds(kmer_test2,v)

print("kmer_test2 k = %s\n\tseed size: %s\n\tseed number: %s\n" % (k, seed_size, seed_nb))

k, seed_size, seed_nb = getSeeds(kmer_test3,v)

print("kmer_test3 k = %s, v = 1\n\tseed size: %s\n\tseed number: %s\n" % (k, seed_size, seed_nb))

kmer_test1 k = 4
	seed size: 2
	seed number: 2

kmer_test2 k = 5
	seed size: 3
	seed number: 2

kmer_test3 k = 6, v = 1
	seed size: 3
	seed number: 2



4\. Créez une fonction `findSeedStarPos` qui, à partir d’un k-mer, d’une taille de seed (`seed_size`), d’un nombre de seeds (`seed_nb`) et d’un suffix tree (`stree`), renvoie toutes les positions possibles des k-mers correspondant à un match.

**Attention :**

* Evitez de compter deux fois un k-mer qui donne un match exact (les deux seeds ne puevent pas être comptés qu'une fois).
* N'oubliez pas de retirer les seeds dont la position de départ est négative.

In [14]:
from textwrap import wrap
def findSeedStarPos(kmer, seed_size, seed_nb, stree):
    """
    Renvoie tous les positions de départ des seeds de k-mer
    entrée kmer : chaîne de caractères avec le motif à analyser
    entrée seed_size : taille de seed calculé
    entrée seed_nb : nombre de seeds contenus dans le k-mer
    entrée stree : sufix tree des séquénces à analyser
    sortie all_candidate_starts : liste de positions de départ qui donnent un match partiel
    """
    
    if len(kmer) < seed_size : return []

    seeds = {s for s in wrap(kmer,seed_size) if len(s) >= seed_size}
    all_candidates = {x for s in seeds for x in stree.find_all(s) if x >= 0}
    candidate_starts = sorted(list({x for x in all_candidates if x >= 0}))
        
    return candidate_starts

In [15]:
kmer = kmer_test1[0]

my_seq = "TACCGCTGATTTT AGGATACATGAGCTCTTGTAGCACTGTTAATA"

# Construire tree
st = STree.STree(my_seq)

# Calculer seeds
k, seed_size, seed_nb # Code

# Trouver positions
all_starts = findSeedStarPos(kmer,seed_size,seed_nb,st)

print(all_starts)

[0, 18]


5\. Développez une fonction qui, à partir d’une liste de positions de départ, génère une liste de k-mers dépassant un seuil établi (`v`). Utilisez la fonction `hamDistance` pour évaluer les correspondances.

**Attention**

* Évitez d'utiliser des k-mers correspondant à un match trop proche de la fin des séquences.
* Ne prenez pas en compte les k-mers candidats qui traversent des séquences séparées par un espace.

In [16]:
def findKmerCandidates(all_candidate_starts, k, kmer, sequences, v):
    """
    entrée all_candidate_starts : liste de positions de départ qui donnent un match partiel
    entrée k : taille de k-mer
    enréee kmer : chaîne de caractères avec le motif à analyser
    entrée sequences : liste de séquences
    entrée v : nombre de variations dans les motifs
    sortie : liste de séquence qui donnent un match avec un nombre inferieur ou égal de variations (v)
    """
    
    all_candidate_kmer = set()
    for c in all_candidate_starts :
        if hamDistance(kmer,sequences[c:c+k]) <= v : 
            all_candidate_kmer.add(sequences[c:c+k])
    return all_candidate_kmer

findKmerCandidates(all_starts, k, kmer, my_seq, v)

{'TACATG', 'TACCGC'}

6\. Définissez la fonction `inexactMatch` qui recherche tous les motifs possibles (k-mers) générés à la question 2 dans le suffix tree construit à partir des séquences contenant le motif variable. La fonction doit renvoyer deux dictionnaires :
* Un dictionnaire contenant les motifs (clés) et leur nombre d'occurrences (valeurs).
* Un autre dictionnaire contenant les motifs (clés) et la liste de toutes leurs variations (valeurs).

In [17]:
def inexactMatch(kmersV, sequences, stree, v):
    """
    cherche de motifs variables dans un suffix tree
    entrée kmersV : liste de motifs à chercher
    entrée sequences : liste de séquences
    entrée stree : suffix tree
    entrée v : nombre de variations dans les motifs
    sortie motif_occur : dictionnaire clés = motif ; value = nombre d'occurrences
    sortie motif_seq : dictionnaire clés = motif ; value = liste de motifs variables
    """

    motif_occur = dict()
    motifs_seq = dict()

    motifs_seq = {}
    for motif in kmersV :
        k, seed_size, seed_nb = getSeeds([motif],v)
        pos = findSeedStarPos(motif,seed_size,seed_nb,stree)
        candidates = findKmerCandidates(pos,k,motif,sequences,v)

        for aux in candidates :
            motif_occur[aux] = len(stree.find_all(aux))
        
        motifs_seq[motif] = candidates
    
    return motif_occur, motifs_seq

seqTest = "banananabanbnaabanbna nbnaaba"

st2 = constructTree(seqTest)

inexactMatch(['banbn', 'anana'], seqTest, st2, 1)

({'banbn': 2, 'banan': 1, 'anaba': 1, 'anana': 2},
 {'banbn': {'banan', 'banbn'}, 'anana': {'anaba', 'anana'}})

7\. Appliquez cette méthode à vos séquences artificielles contenant le motif variable. Ensuite, affichez le motif variable le plus fréquent ainsi que toutes ses variations dans votre nouveau jeu de données artificiel.

In [18]:
motif_occur_var, motif_occur_var_seqs = inexactMatch([motif],seqs,tree,1)

# Top motifs
top_motifs = getTopMotifs(motif_occur_var,10,True)
print("les tops 10 motifs")
for x in top_motifs : print("--",x,":",top_motifs[x])

print("\nles variations")
for x in top_motifs : 
    if x in motif_occur_var_seqs : print("--",x,":",motif_occur_var_seqs[x])
    else : print("--",x,": pas de variation")

les tops 10 motifs
-- ACAT : 12
-- ACAA : 2
-- ACCT : 2
-- ACAC : 2
-- ACGT : 1

les variations
-- ACAT : {'ACAA', 'ACAT', 'ACGT', 'ACCT', 'ACAC'}
-- ACAA : pas de variation
-- ACCT : pas de variation
-- ACAC : pas de variation
-- ACGT : pas de variation


In [19]:
# Tester gather motifs
motif_occur_var_gat = gatherRevCompMotifs(motif_occur_var)

# Top motifs
top_motifs_gat = getTopMotifs(motif_occur_var_gat,10,True)
print("les tops 10 motifs")
for x in top_motifs_gat : print("--",x,":",top_motifs_gat[x])

print("\nles variations")
for x in top_motifs_gat : 
    if x in motif_occur_var_seqs :
        print("--",x,":",motif_occur_var_seqs[x])

les tops 10 motifs
-- ACAT : (12, 0)
-- ACAA : (2, 0)
-- ACCT : (2, 0)
-- ACAC : (2, 0)
-- ACGT : (1, 1)

les variations
-- ACAT : {'ACAA', 'ACAT', 'ACGT', 'ACCT', 'ACAC'}


8\. Générez un logo de motif à partir des séquences du meilleur motif variable détecté. Vous pouvez utiliser ce site : [WebLogo](https://weblogo.berkeley.edu/logo.cgi). Affichez votre LOGO ci-dessous.

In [20]:
kmer = "ACAT"

<font color='blue'>
Markdown
</font>

9\. Avez-vous réussi à retrouver le motif initialement implanté dans les séquences ? L'algorithme était-il rapide ? Quelle est la complexité de chaque recherche de motif ?

<font color='blue'>
Votre reponse
</font>

10\. Testez l'algorithme suffix tree sur vos données de ChIP-Seq, puis générez le LOGO du motif trouvé.

In [ ]:
# Pensé à créer plusieur tests avec differentes tailles de séquences
# Code

In [ ]:
# Sauvarger kmers variables

kmer # Code

out_seqs_motifs_file = "motifs_" + kmer + "_list.txt"

# Code

<font color='blue'>
Markdown
</font>